# 異常検知（Anomaly Detection）
### 外れ値検知と正規性検定
#### Davisデータ使用
Google search: "R dataset davis"<br>
The Davis data frame has 200 rows and 5 columns. The subjects were men and women engaged in regular exercise. There are some missing data.<br>
- weight:Measured weight in kg.
- height:Measured height in cm.
- repwt:Reported weight in kg.
- repht:Reported height in cm.

weightは欠損値なし、repwt, rephtは欠損値あり<br>

References
- Davis, C. (1990) Body image and weight preoccupation: A comparison between exercising and non-exercising women. Appetite, 15, 13--21.
- Fox, J. (2008) Applied Regression Analysis and Generalized Linear Models, Second Edition. Sage.
- Fox, J. and Weisberg, S. (2011) An R Companion to Applied Regression, Second Edition, Sage.

2022年12月時点で次からデータを取得<br>
- https://vincentarelbundock.github.io/Rdatasets/datasets.html
- https://www.picostat.com/dataset/r-dataset-package-car-davis

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('dataDavis.csv')
print(df.head(5))
data = df['weight'].to_numpy() # pandas to numpy.ndarray
N = len(data)
print('N=',N, ' type(data)',type(data))

基本統計量を求める。不偏標準偏差とするためstd(ddof=1)としていることに注意。<br>
下記より，平均値から最小値，最大値までの距離がかなり異なることがわかる。

In [ ]:
mean, median, std = data.mean(), np.median(data),  data.std(ddof=1)
print("mean=",mean, " median=",median," std=",std)
print("min=",data.min(), " max=",data.max())
print("|min - mean|=",np.abs(data.min()-mean),"  |mean - max|=",np.abs(mean-data.max()))

### ヒストグラム
matplotlib.pyplot.hist https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html<br>

### カーネル密度推定(KDE:kernel density estimation)
https://ja.wikipedia.org/wiki/カーネル密度推定<br>
KDE関数を提供しているのは，SciPy, seaborn<br>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html<br>
https://seaborn.pydata.org/generated/seaborn.kdeplot.html<br>
他に，pandas, statsmodels, sklearnも提供している。

In [ ]:
X = np.arange(0.0, 160.0, 0.1) # range according to min and max
pdf = stats.norm.pdf(X, loc=data.mean()-5.0, scale=data.std(ddof=1))
kde = stats.gaussian_kde(data)

In [ ]:
fig = plt.subplots(figsize=(5,3))
freq, bins, temp = plt.hist(data, bins=50, density=True, color=(0.6, 0.6, 0.6), label='data')
plt.plot(X, pdf, linewidth=2, color='k', label='pdf')
plt.plot(X, kde(X), linestyle='dashed', color='k', label='kde')
#
plt.title("Histgram, pdf, kde")
plt.xlabel('weight [kg]')
plt.ylabel('freq., prob.')
plt.legend()
plt.grid(True)

#plt.savefig('fig_STA_AnomarlyDetectionHist.pdf', bbox_inches='tight')
plt.show()

## 外れ値の検知

#### 3$\sigma$法による外れ値検出

In [ ]:
coef = 3
lower, upper = mean - coef*std, mean + coef*std
outlier = [val for val in data if (val < lower) or (val > upper) ]
print("interval =[",lower, ",", upper," ], outlier=",outlier)

#### 箱ひげ図による外れ値検出
matplotlib.pyplot.boxplot https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.boxplot.html

In [ ]:
fig = plt.subplots(figsize=(3,3))

plt.boxplot(data, whis=1.5) # whis is whisker
#plt.title('box plot')
plt.ylabel('weight')
plt.grid()

#plt.savefig('fig_STA_AnomarlyDetectionBoxPlot.pdf', bbox_inches='tight')
plt.show()

### スミルノフ・グラブス検定(Smirnov Grubbs Test)
検定統計量（$T$: test statistics）
$$  T = \frac{|x_i - \hat{\mu}|}{\hat{\sigma}} $$

次の基準値（critical value）と比較する。
$$
c_{val} = \frac{(N-1)T_{\alpha}}{\sqrt{N(N-2) + N T_{\alpha}^2}}
$$
ここに，
$$
t_{\alpha} = t \left( \alpha/N, N-2 \right) \rm{\;\;for \;\; one-sided \; test}
$$
$$
t_{\alpha} = t \left( \alpha/(2N), N-2 \right) \rm{\;\;for \;\; two-sided \; test}
$$

仮説は次のように置く。<br>
H0:	There are no outliers in the data set<br>
H1:	There is exactly one outlier in the data set<br>

#### 参考サイト
- https://en.wikipedia.org/wiki/Grubbs%27s_test
- The critical value for the test is   -->  Cval https://www.real-statistics.com/students-t-distribution/identifying-outliers-using-t-distribution/grubbs-test/
- G critical value https://www.statisticshowto.com/grubbs-test/
- https://en.wikipedia.org/wiki/Nikolai_Smirnov_(mathematician)

In [ ]:
def funSGtest(alpha, data):
    N = len(data)
#    print('n=', n)
    mean, std = np.mean(data), np.std(data, ddof=1)
    id_min, id_max = np.argmin(data), np.argmax(data)
    id = id_max \
        if np.abs(data[id_max]-mean) > np.abs(data[id_min]-mean) \
        else id_min
    T = np.abs(data[id]-mean)/std # test statistics
    
    talp = stats.t.isf((alpha/N) / 2, N-2)  # two‐sided test
    cval = (N-1)*talp/np.sqrt(N*(N-2) + N*talp*talp) # critical value
#    print(t_stat, cval, t)

    flag = T > cval # True:H0 rejected, False:H0 not rejected
    return (flag, id)

In [ ]:
alpha = 0.05
flag, id = funSGtest(alpha, data)
print(flag, 'data[',id,']=',data[id])

data2 = np.delete(data, id)
flag, id2 = funSGtest(alpha, data2)
print(flag, 'data[',id2,']=',data2[id2])

data3 = np.delete(data2, id2)
flag, id3 = funSGtest(alpha, data3)
print(flag, 'data[',id3,']=',data3[id3])

#### スミルノフ・グラブス検定のパッケージ
下記にあるが，両側検定では，どの値が除去されたかわかりづらいことに注意<br>
また，別途，インストールが必要<br>
https://pypi.org/project/outlier_utils/

In [ ]:
from outliers import smirnov_grubbs as grubbs
grubbs.max_test_outliers(data, alpha=0.05) # one-side

## 正規性の検定

#### Q-Qプロット（Quantile-Quantile Plot）
2つの確率分布を比較するための図です。2つの分布が類似している場合には、プロットが一直線に並ぶ。
#### statsmodels.graphics.gofplots.qqplot
https://www.statsmodels.org/dev/generated/statsmodels.graphics.gofplots.qqplot.html<

In [ ]:
fig = sm.qqplot(data, dist=stats.norm, line='s')
plt.grid()
#plt.savefig('fig_STA_AnomarlyDetectionQQPlot.pdf', bbox_inches='tight')

### シャピロ・ウィルク検定(Shapiro-Wilk test, SW検定)　
https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test<br>
データ数が少ない時（例えば1,000データ以下）に有効，また，正規分布以外にも適用可
#### scipy.stats.shapiro
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html<br>

H0: 帰無仮説「母集団が正規分布に従う」<br>
H1: 対立仮説「母集団が正規分布に従わない」<br>

In [ ]:
stats.shapiro(data)

p < 0.05 より、H0は棄却、H1を採択

In [ ]:
stats.shapiro(data2)

In [ ]:
stats.shapiro(data3)

## 考察：
#### 外れ値検知
3$\sigma$法では２つ，箱ひげ図は3つの外れ値（ただし、whis=1.5）を示した。前者は標準正規分布を仮定，後者は確率分布を仮定していない。<br>
SG検定は，大きい方からの２つの値は外れ値、3番目に大きい値は帰無仮説を否定できなかったので何とも言えない。<br>
#### 正規性の検定
Q-Qプロット：視覚的には正規性でないように認められる。<br>
SW検定：data，および，このうちの外れ値と考えられる大きな値を1～2つ除外してもpが十分小さいため，データは正規分布ではないとの結果を得た。

## 例：人工データ
次の人工データ（artificial data）を用いた考察を行う。

In [ ]:
np.random.seed(123)
N = 200
#N = 100
m0, std0 = 20, 5
x = np.arange(5, 35, 0.1)
adata = stats.norm.rvs(loc=m0, scale=std0, size=N)
pdf = stats.norm.pdf(x, loc=m0, scale=std0)
cdf = stats.norm.cdf(x, loc=m0, scale=std0)

In [ ]:
freq, bins, temp = plt.hist(adata, bins=50, density=True, color=(0.6, 0.6, 0.6), label='data')
plt.plot(x, pdf, linewidth=2, color='k', label='pdf')
plt.plot(x, kde(x), linestyle='dashed', color='b', label='kde')
#
plt.title("test")
plt.xlabel('x')
plt.ylabel('freq., prob.')
plt.legend()
plt.grid(True)

In [ ]:
fig = plt.subplots(figsize=(5,3))

plt.boxplot(adata, whis=1.5)
plt.grid()

In [ ]:
alpha = 0.05
id = np.argmin(adata)
print(N, id, adata[id])

x_stat = np.abs(adata[id]-adata.mean())/adata.std(ddof=1)
t = stats.t.isf((alpha/N) / 2, N-2)  # Divide alpha by 2, since two‐sided test
SGtest = (N-1)*t/np.sqrt(N*(N-2)+N*t*t)
print(x_stat, SGtest, t)
if x_stat > SGtest:
    print('H0 be rejected，H1 be accepted，thus it is outlier.')
else:
    print('H0 not be rejected.')

In [ ]:
stats.shapiro(adata)

人工データの場合，箱ひげ図は外れ値が小さい値で一つあると示した。<br>
一歩，SG検定，SW検定共に，H0を棄却できなかった。もちろん，この結果は「正規性がある」とは言ってはいないが、「正規性でない」とも言っていないので正規性の可能性はある。<br>
ここで、正規性であると仮定したとき、正規分布は非常に大きい値（または小さい値）を取ることができる。もちろん、その値を取る確率は非常に小さい。<br>
このことから、外れ値があったとしても、正規分布を仮定するならば、その値を直ちに除外することはできないことを意味する。